:::{note} Tutorial 6. **Structuring Codebases**
:icon: false

#### Classes and Design Patterns

Once you go beyond a simple dashboard or data app to a full fleshed application organizing your code and reasoning about state become a lot more difficult. In this section we will explore some common design patterns for structuring the code for your larger application.

:::

In [ ]:
import param
import pandas as pd
import panel as pn

pn.extension('tabulator', 'vega', throttled=True)

## Composing Parameterized Classes

When building a complex application object-oriented programming usually suggests to build compositional classes, each with a well defined role. Let us explore one such pattern:

In [ ]:
from panel.viewable import Viewer

CARD_STYLE = """
:host {{
  box-shadow: rgba(50, 50, 93, 0.25) 0px 6px 12px -2px, rgba(0, 0, 0, 0.3) 0px 3px 7px -3px;
  padding: {padding};
}} """

class DataStore(Viewer):
    
    data = param.DataFrame()

    filters = param.List(constant=True)

    def __init__(self, **params):
        super().__init__(**params)
        dfx = self.param.data.rx()
        widgets = []
        for filt in self.filters:
            dtype = self.data.dtypes[filt]
            if dtype.kind == 'f':
                widget = pn.widgets.RangeSlider(name=filt, start=dfx[filt].min(), end=dfx[filt].max())
                condition = dfx[filt].between(*widget.rx())
            else:
                options = dfx[filt].unique().tolist()
                widget = pn.widgets.MultiChoice(name=filt, options=options)
                condition = dfx[filt].isin(widget.rx().rx.where(widget, options))
            dfx = dfx[condition]
            widgets.append(widget)
        self.filtered = dfx
        self.count = dfx.rx.len()
        self.total_capacity = dfx.t_cap.sum()
        self.avg_capacity = dfx.t_cap.mean()
        self.avg_rotor_diameter = dfx.t_rd.mean()
        self.top_manufacturers = dfx.groupby('t_manu').p_cap.sum().sort_values().iloc[-10:].index.to_list()
        self._widgets = widgets

    def filter(self, ):
        return 
        
    def __panel__(self):
        return pn.Column('## Filters', *self._widgets, stylesheets=[CARD_STYLE.format(padding='5px 10px')], margin=10)
        
class View(Viewer):

    data_store = param.ClassSelector(class_=DataStore)

Here we declared a `DataStore` class that will be responsible for loading some data and transforming it in various ways and a `View` class that holds a reference to the `DataStore` as a parameter. Now we can have any number of concrete `View` classes that consume data from the DataStore and render it in any number of ways:

In [ ]:
import altair as alt
import holoviews as hv
import hvplot.pandas

class Table(View):
    
    columns = param.List(default=['p_name', 'p_year', 't_state', 't_county', 't_manu', 't_cap', 'p_cap'])
    
    def __panel__(self):
        data = self.data_store.filtered[self.param.columns]
        return pn.widgets.Tabulator(
            data, pagination='remote', page_size=13, stylesheets=[CARD_STYLE.format(padding='10px')],
            margin=10,
        )
    
class Histogram(View):

    def __panel__(self):
        df = self.data_store.filtered
        df = df[df.t_manu.isin(self.data_store.top_manufacturers)]
        fig = pn.rx(alt.Chart)((df.rx.len()>5000).rx.where(df.sample(5000), df),  title='Capacity by Manufacturer').mark_circle(size=8).encode(
            y="t_manu:N",
            x="p_cap:Q",
            yOffset="jitter:Q",
            color=alt.Color('t_manu:N').legend(None)
        ).transform_calculate(
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        ).properties(
            height=400,
            width=600,
        )
        return pn.pane.Vega(fig, stylesheets=[CARD_STYLE.format(padding='0')], margin=10)
    
class Indicators(View):
    
    def __panel__(self):
        style = {'stylesheets': [CARD_STYLE.format(padding='10px')]}
        return pn.FlexBox(
            pn.indicators.Number(
                value=self.data_store.total_capacity / 1e6, name='Total Capacity (TW)', format='{value:,.2f}', **style
            ),
            pn.indicators.Number(
                value=self.data_store.count, name='Count', format='{value:,.0f}', **style
            ),
            pn.indicators.Number(
                value=self.data_store.avg_capacity , name='Avg. Capacity (kW)', format='{value:,.2f}', **style
            ),
            pn.indicators.Number(
                value=self.data_store.avg_rotor_diameter, name='Avg. Rotor Diameter (m)', format='{value:,.2f}', **style
            ),
            pn.indicators.Number(
                value=self.data_store.avg_rotor_diameter, name='Avg. Rotor Diameter (m)', format='{value:,.2f}', **style
            ),
            
        )

Lastly we can create an `App` class that coordinates the creation of the `View` components and lays them out on the page.

In [ ]:
class App(Viewer):
    
    data_store = param.ClassSelector(class_=DataStore)
    
    title = param.String()
    
    views = param.List()
    
    def __init__(self, **params):
        super().__init__(**params)
        updating = self.data_store.filtered.rx.updating()
        updating.rx.watch(lambda updating: pn.state.curdoc.hold() if updating else pn.state.curdoc.unhold()) 
        self._views = pn.FlexBox(*(view(data_store=self.data_store) for view in self.views), loading=updating)
        self._template = pn.template.MaterialTemplate(title=self.title)
        self._template.sidebar.append(self.data_store)
        self._template.main.append(self._views)
        
    def servable(self):
        if pn.state.served:
            return self._template.servable()
        return self
        
    def __panel__(self):
        return pn.Row(self.data_store, self._views)

Now let's see what we have built:

In [ ]:
ds = DataStore(data=pd.read_parquet('./windturbines.parq'), filters=['p_year', 'p_cap', 't_manu'])

app = App(data_store=ds, views=[Indicators, Histogram, Table], title='Windturbine Explorer')

app.servable()